In [1]:
import pandas
import geopandas 
import feedparser
import requests
from   zipfile import ZipFile
import datetime 
from   local_module import NhcDownloaderBot

%matplotlib inline

In [2]:
url = 'https://www.nhc.noaa.gov/index-at.xml'
f = feedparser.parse(url)
f

{'bozo': True,
 'entries': [{'title': 'Atlantic Tropical Weather Outlook',
   'title_detail': {'type': 'text/plain',
    'language': None,
    'base': 'https://www.nhc.noaa.gov/index-at.xml',
    'value': 'Atlantic Tropical Weather Outlook'},
   'summary': '<br />\n000<br />\nABNT20 KNHC 282344<br />\nTWOAT <br />\n<br />\nTropical Weather Outlook<br />\nNWS National Hurricane Center Miami FL<br />\n800 PM EDT Tue Jun 28 2022<br />\n<br />\nFor the North Atlantic...Caribbean Sea and the Gulf of Mexico:<br />\n<br />\nEast of the Windward Islands:<br />\nThe National Hurricane Center is issuing advisories on Potential <br />\nTropical Cyclone Two, located near the southern Windward Islands.<br />\n* Formation chance through 48 hours...high...70 percent.<br />\n* Formation chance through 5 days...high...90 percent.<br />\n<br />\nNorthern Gulf of Mexico:<br />\nAn area of low pressure centered over the northwestern Gulf of <br />\nMexico continues to produce disorganized showers and thun

In [3]:
#f
#f.keys()
# len(f.entries)
# f['entries'][1]['nhc_atcf']
# f.status
# f.feed
# f.feed['subtitle']
# f.entries
# type(f.entries)
# for entry in f.entries:
#     print(entry.title)

In [3]:
df = pandas.DataFrame(f.entries)
df = df.drop(columns=['title_detail', 'summary', 'summary_detail', 'published_parsed', 'links', 'link', 'id', 
                      'guidislink', 'authors', 'author', 'author_detail'])
df.head(5)

,title,published,nhc_center,nhc_type,nhc_name,nhc_wallet,nhc_atcf,nhc_datetime,nhc_movement,nhc_pressure,nhc_wind,nhc_headline,nhc_cyclone
0,Atlantic Tropical Weather Outlook,"Tue, 28 Jun 2022 23:45:02 GMT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Summary for Potential Tropical Cyclone Two (AT...,"Wed, 29 Jun 2022 00:24:27 GMT","10.4, -61.0",Potential Tropical Cyclone,Two,AT2,AL022022,8:00 PM AST Tue Jun 28,W at 25 mph,1009 mb,40 mph,...DISTURBANCE NOW MOVING THROUGH THE SOUTHERN...,
2,Potential Tropical Cyclone Two Public Advisory...,"Wed, 29 Jun 2022 00:24:27 GMT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Potential Tropical Cyclone Two Forecast Adviso...,"Tue, 28 Jun 2022 20:42:53 GMT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Potential Tropical Cyclone Two Forecast Discus...,"Tue, 28 Jun 2022 20:43:39 GMT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
tc_list = [nhc_atcf[2:4] for nhc_atcf in df['nhc_atcf'] if pandas.isnull(nhc_atcf) == False] 
tc_list

['02']

In [5]:
#{key: value for (key, value) in iterable}
d = {nhc_atcf: [nhc_name    , nhc_type, nhc_center, nhc_movement, 
                nhc_pressure, nhc_wind, published , nhc_datetime]         
     for (nhc_name    , nhc_atcf, nhc_type  , nhc_center  , nhc_movement,
          nhc_pressure, nhc_wind, published , nhc_datetime) 
     in  zip(df['nhc_name']    , df['nhc_atcf']    , df['nhc_type'], df['nhc_center'], 
             df['nhc_movement'], df['nhc_pressure'], df['nhc_wind'], df['published'],
             df['nhc_datetime']) 
     if pandas.isnull(nhc_name) == False}   
d

{'AL022022': ['Two',
  'Potential Tropical Cyclone',
  '10.4, -61.0',
  'W at 25 mph',
  '1009 mb',
  '40 mph',
  'Wed, 29 Jun 2022 00:24:27 GMT',
  '8:00 PM AST Tue Jun 28']}

In [6]:
class NhcRssParser:
    def __init__(self, url):    
        self.url   = url 
        self.f     = f   =  feedparser.parse(self.url)
        self.df    = df  =  pandas.DataFrame(self.f.entries).drop(columns=['title_detail', 'summary', 'summary_detail', 'published_parsed', 'links', 'link', 'id', 
                               'guidislink', 'authors', 'author', 'author_detail'])
    
    def tc_list(self):
        return [nhc_atcf for nhc_atcf in self.df['nhc_atcf'] if pandas.isnull(nhc_atcf) == False]  
    #{key: value for (key, value) in iterable}
    def tc_dict(self):
        return {nhc_atcf: [nhc_name    , nhc_type, nhc_center, nhc_movement, 
                        nhc_pressure, nhc_wind, published , nhc_datetime]         
             for (nhc_name    , nhc_atcf, nhc_type  , nhc_center  , nhc_movement,
                  nhc_pressure, nhc_wind, published , nhc_datetime) 
             in  zip(self.df['nhc_name']    , self.df['nhc_atcf']    , self.df['nhc_type'], self.df['nhc_center'], 
                     self.df['nhc_movement'], self.df['nhc_pressure'], self.df['nhc_wind'], self.df['published'],
                     self.df['nhc_datetime']) 
             if pandas.isnull(nhc_name) == False}

url = 'https://www.nhc.noaa.gov/index-at.xml'
a = NhcRssParser(url)
a.df

list_ = a.tc_list()
dict_ = a.tc_dict()
for i in range(len(list_)):
    print(dict_[list_[i]][2])


10.4, -61.0


In [8]:
tcdictgral = {
             'nhc_name':    [],
             'nhc_type':    [['tropical depresion', 'tropical storm', 'hurricane', 'major hurricane', 'remanents'],
                             ['DT'                , 'TT'            , 'huracan'  , 'HM'             , 'remanentes']], 
             'nhc_center':  [], 
             'nhc_movement':['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW'], 
             'nhc_pressure':[], 
             'nhc_wind':    [], 
             'published':   [], 
             'nhc_datetime':[]
             }

In [13]:
class NhcRssParser:
    def __init__(self, url):    
        self.url   = url 
        self.f     = f   =  feedparser.parse(self.url)
        self.df    = df  =  pandas.DataFrame(self.f.entries).drop(columns=['title_detail', 'summary', 'summary_detail', 'published_parsed', 'links', 'link', 'id', 
                               'guidislink', 'authors', 'author', 'author_detail'])
        self.tcdictgral  = tcdictgral = {
                     'nhc_name':    [],
                     'nhc_type':    [['tropical depression', 'tropical storm', 'hurricane', 'major hurricane', 'remanents', 'potential tropical cyclone'],
                                     ['DT'                , 'TT'            , 'huracan'  , 'HM'             , 'remanentes', 'PT']], 
                     'nhc_center':  [], 
                     'nhc_movement':['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW'], 
                     'nhc_pressure':[], 
                     'nhc_wind':    [], 
                     'published':   [], 
                     'nhc_datetime':[]
                     }
        self.numbers_in_english = ['ONE', 'TWO', 'THREE', 'FOUR', 'FIVE', 'SIX', 'SEVEN', 'EIGHT', 'NINE', 'TEN', 'ELEVEN',
                                  'TWELVE', 'THIRTEEN', 'FOURTEEN', 'FIFTEEN', 'SIXTEEN', 'SEVENTEEN', 'EIGHTEEN', 'NINETEEN',
                                  'TWENTY']
        self.numbers_in_spanish = ['UNO', 'DOS', 'TRES', 'CUATRO', 'CINCO', 'SEIS', 'SIETE', 'OCHO', 'NUEVE', 'DIEZ', 'ONCE',
                                  'DOCE', 'TRECE', 'CATORCE', 'QUINCE', 'DIE', 'SEVENTEEN', 'EIGHTEEN', 'NINETEEN',
                                  'TWENTY']
    
    #{key: value for (key, value) in iterable}
    def tc_dict_list(self):
        tclist = [nhc_atcf for nhc_atcf in self.df['nhc_atcf'] if pandas.isnull(nhc_atcf) == False] 
        tcdict = {nhc_atcf: [nhc_name    , nhc_type, nhc_center, nhc_movement, 
                             nhc_pressure, nhc_wind, published , nhc_datetime]         
             for (nhc_name    , nhc_atcf, nhc_type  , nhc_center  , nhc_movement,
                  nhc_pressure, nhc_wind, published , nhc_datetime) 
             in  zip(self.df['nhc_name']    , self.df['nhc_atcf']    , self.df['nhc_type'], self.df['nhc_center'], 
                     self.df['nhc_movement'], self.df['nhc_pressure'], self.df['nhc_wind'], self.df['published'],
                     self.df['nhc_datetime']) 
             if pandas.isnull(nhc_name) == False}
        for key in tcdict.keys():
            tcdict[key][1] = self.tcdictgral['nhc_type'][1][self.tcdictgral['nhc_type'][0].index(tcdict[key][1].casefold())]
            tcdict[key][0] = tcdict[key][0].upper()
            tcdict[key][3] = tcdict[key][3].replace('W', 'O')
            if tcdict[key][0] in self.numbers_in_english:
                tcdict[key][0] = self.numbers_in_spanish[self.numbers_in_english.index(tcdict[key][0])]                
            try:
                lat  = float(tcdict[key][2][:4])
                lon  = float(tcdict[key][2][6:])
            except ValueError: 
                lat  = float(tcdict[key][2][:3])
                lon  = float(tcdict[key][2][5:])
            tcdict[key][2] = [lat, lon]
         
        return tcdict, tclist
        
url    = 'https://www.nhc.noaa.gov/index-at.xml'
tcdict, tclist = NhcRssParser(url).tc_dict_list()
tcdict

# for i in range(len(tclist)):
#     tcdict[tclist[i]][1] = tcdictgral['nhc_type'][1][tcdictgral['nhc_type'][0].index(tcdict[tclist[i]][1].casefold())]
#     tcdict[tclist[i]][0] = tcdict[tclist[i]][0].upper()
#     tcdict[tclist[i]][3] = tcdict[tclist[i]][3].replace('W', 'O')
#     try:
#         lat  = float(tcdict[tclist[i]][2][:4])
#         lon  = float(tcdict[tclist[i]][2][6:])
#     except ValueError:
#         lat  = float(tcdict[tclist[i]][2][:3])
#         lon  = float(tcdict[tclist[i]][2][5:])
#     tcdict[tclist[i]][2] = [lat, lon]

{'AL022022': ['DOS',
  'PT',
  [10.9, -62.8],
  'ONO at 26 mph',
  '1011 mb',
  '40 mph',
  'Wed, 29 Jun 2022 02:48:31 GMT',
  '11:00 PM AST Tue Jun 28']}

In [11]:
numbers_in_english = ['ONE', 'TWO', 'THREE', 'FOUR', 'FIVE', 'SIX', 'SEVEN', 'EIGHT', 'NINE', 'TEN', 'ELEVEN',
                                  'TWELVE', 'THIRTEEN', 'FOURTEEN', 'FIFTEEN', 'SIXTEEN', 'SEVENTEEN', 'EIGHTEEN', 'NINETEEN',
                                  'TWENTY']
numbers_in_spanish = ['UNO', 'DOS', 'TRES', 'CUATRO', 'CINCO', 'SEIS', 'SIETE', 'OCHO', 'NUEVE', 'DIEZ', 'ONCE',
                                  'DOCE', 'TRECE', 'CATORCE', 'QUINCE', 'DIE', 'SEVENTEEN', 'EIGHTEEN', 'NINETEEN',
                                  'TWENTY']

nombre = 'ONE'
nombre = numbers_in_spanish[numbers_in_english(tcdict[key][0])]   
nombre

NameError: name 'key' is not defined

In [11]:
class NhcDownloaderBot:
    def __init__(self, storm_number, year):
        self.storm_number = storm_number
        self.year         = year
        
        
        self.file_names     = file_names = [
                            f'al{self.storm_number}{self.year}_5day_latest.zip',
                            f'al{self.storm_number}{self.year}_fcst_latest.zip',
                            f'al{self.storm_number}{self.year}_best_track.zip',
                            f'gtwo_shapefiles.zip',
                            f'wsp_120hrhalfDeg_latest.zip',
                            f'wsp_120hr5km_latest.zip'
                                           ]
        self.urls           = urls = [
                            f'https://www.nhc.noaa.gov/gis/forecast/archive',
                            f'https://www.nhc.noaa.gov/gis/forecast/archive',
                            f'https://www.nhc.noaa.gov/gis/best_track',
                            f'https://www.nhc.noaa.gov/xgtwo',
                            f'https://www.nhc.noaa.gov/gis/forecast/archive',
                            f'https://www.nhc.noaa.gov/gis/forecast/archive'
                                    ]
        self.gdf_names      = gdf_names = {
                            file_names[0] : ['track_line_gdf', 'cone_gdf', 'points_gdf'],
                            file_names[1] : ['init_radii_gdf', 'fcst_radii_gdf'],
                            file_names[2] : ['best_track_points_gdf', 'best_track_line_gdf', 'best_track_radii_gdf', 'best_track_swath_gdf'],
                            file_names[3] : ['gtwo_areas_gdf', 'gtwo_lines_gdf', 'gtwo_points_gdf'],
                            file_names[4] : ['wsp_34_gdf_points', 'wsp_50_gdf_points', 'wsp_64_gdf_points'],
                            file_names[5] : ['wsp_34_gdf_polygons', 'wsp_50_gdf_polygons', 'wsp_64_gdf_polygons']
                                          }
    def nhc_gis_downloader(self, file_names, urls, gdf_names):

        for idx, file_name in enumerate(file_names):
            url = f'{urls[idx]}/{file_name}'
            r = requests.get(url)
            with open(f'nhc_latest/{file_name}', 'wb') as code:
                code.write(r.content) 
        _  = [0, 3, 4, 5]  
        __ = [2, 7, 12]  
        for idx, file_name in enumerate([file_names[0],  file_names[3], 
                                        file_names[4],  file_names[5]]): 
            with ZipFile(f'nhc_latest/{file_name}', 'r') as zip_file_name:
                for idx2 in range(len(gdf_names[file_names[_[idx]]])):
                    gdf_names[file_names[_[idx]]][idx2]  = geopandas.read_file(
                    f'zip://./nhc_latest/{file_name}!{zip_file_name.namelist()[__[idx2]]}')
        _  = [0,5]                                 
        with ZipFile(f'nhc_latest/{file_names[1]}', 'r') as zip_file_name:
                for idx in range(2):
                    gdf_names[file_names[1]][idx]  = geopandas.read_file(
                    f'zip://./nhc_latest/{file_names[1]}!{zip_file_name.namelist()[_[idx]]}')
        _  = [2, 7, 13, 16]                                 
        with ZipFile(f'nhc_latest/{file_names[2]}', 'r') as zip_file_name:
            for idx in range(4):
                try:
                    gdf_names[file_names[2]][idx]  = geopandas.read_file(
                    f'zip://./nhc_latest/{file_names[2]}!{zip_file_name.namelist()[_[idx]]}')
                except IndexError:
                    gdf_names[file_names[2]][idx] = None
                    
        return gdf_names
    
    @classmethod
    def gdf_file(cls, storm_number, year):
        cls._                     = NhcDownloaderBot(storm_number, year)
        cls.track_line_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[0]][0]
        cls.cone_gdf              = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[0]][1]
        cls.points_gdf            = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[0]][2]
        cls.init_radii_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[1]][0]
        cls.fcst_radii_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[1]][1]
#         cls.best_track_points_gdf = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[2]][0]
#         cls.best_track_line_gdf   = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[2]][1]
#         cls.best_track_radii_gdf  = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[2]][2]
#         cls.best_track_swath_gdf  = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[2]][3]
        cls.gtwo_areas_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[3]][0]
        cls.gtwo_lines_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[3]][1]
        cls.gtwo_points_gdf       = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[3]][2]
        cls.wsp_34_gdf_points     = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[4]][0]
        cls.wsp_50_gdf_points     = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[4]][1]
        cls.wsp_64_gdf_points     = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[4]][2]
        cls.wsp_34_gdf_polygons   = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[5]][0]
        cls.wsp_50_gdf_polygons   = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[5]][1]
        cls.wsp_64_gdf_polygons   = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[5]][2]

In [12]:
NhcDownloaderBot.gdf_file(14, 2020)
print('Done')

DriverError: '/vsizip/./nhc_latest/al142020_best_track.zip/AL142020_windswath.shp.xml' not recognized as a supported file format.

In [7]:
NhcDownloaderBot.best_track_swath_gdf

,RADII,STORMID,BASIN,STORMNUM,ADVNUM,SYNOPTIME,TIMEZONE,NE,SE,SW,NW,geometry
0,34.0,AL142021,al,14.0,12.0,2021091212,UTC,90.0,0.0,0.0,0.0,"POLYGON ((-95.18587 22.49726, -95.15759 22.496..."
1,34.0,AL142021,al,14.0,12.0,2021091218,UTC,90.0,0.0,0.0,0.0,"POLYGON ((-95.58572 23.89726, -95.55714 23.896..."
2,34.0,AL142021,al,14.0,12.0,2021091300,UTC,100.0,0.0,0.0,0.0,"POLYGON ((-96.08392 25.56363, -96.05173 25.563..."
3,34.0,AL142021,al,14.0,12.0,2021091306,UTC,100.0,80.0,0.0,0.0,"POLYGON ((-96.48375 26.76363, -96.45123 26.763..."
4,50.0,AL142021,al,14.0,12.0,2021091306,UTC,30.0,0.0,0.0,0.0,"POLYGON ((-96.49517 25.59909, -96.48552 25.598..."
5,34.0,AL142021,al,14.0,12.0,2021091312,UTC,100.0,80.0,0.0,0.0,"POLYGON ((-96.88371 27.06363, -96.85111 27.063..."
6,50.0,AL142021,al,14.0,12.0,2021091312,UTC,30.0,0.0,0.0,0.0,"POLYGON ((-96.89516 25.89909, -96.88548 25.898..."
7,34.0,AL142021,al,14.0,12.0,2021091318,UTC,100.0,100.0,0.0,0.0,"POLYGON ((-96.48347 28.56363, -96.45042 28.563..."
8,50.0,AL142021,al,14.0,12.0,2021091318,UTC,30.0,0.0,0.0,0.0,"POLYGON ((-96.49510 27.39909, -96.48528 27.398..."
9,34.0,AL142021,al,14.0,12.0,2021091400,UTC,100.0,120.0,40.0,50.0,"POLYGON ((-95.98328 29.76363, -95.94984 29.763..."


In [ ]:
ct = []
for tc in tc_list:
    df = NhcDownloaderBot(tc, 2021)
    df = df.gdf_file(tc, 2021)
    ct.append(NhcDownloaderBot.track_line_gdf)
    if tc == tc_list[-1]:
        print(f'--------------------done-----------------------')
    
#     NhcDownloaderBot.init_radii_gdf
    

In [ ]:
ct[0]

In [1]:
import pandas
import geopandas 
from   pyproj import CRS
import matplotlib.pyplot as plt
from   matplotlib.pyplot import imread
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.ticker as mticker
import matplotlib.patheffects as path_effects
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from   cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from   shapely.geometry import Point, LineString, Polygon, MultiPoint

import requests
from   zipfile import ZipFile
import datetime 
import feedparser
import math

class NhcDownloaderBotNEW:
    def __init__(self, storm_number = 1, year = 2020):
        self.file_names   = file_names = [
                                            f'al{storm_number}{year}_5day_latest.zip',
                                            f'al{storm_number}{year}_fcst_latest.zip',
                                            f'al{storm_number}{year}_best_track.zip',
                                            f'gtwo_shapefiles.zip',
                                            f'wsp_120hrhalfDeg_latest.zip',
                                            f'wsp_120hr5km_latest.zip'
                                         ]
        self.urls         = urls       = [
                                            f'https://www.nhc.noaa.gov/gis/forecast/archive',
                                            f'https://www.nhc.noaa.gov/gis/forecast/archive',
                                            f'https://www.nhc.noaa.gov/gis/best_track',
                                            f'https://www.nhc.noaa.gov/xgtwo',
                                            f'https://www.nhc.noaa.gov/gis/forecast/archive',
                                            f'https://www.nhc.noaa.gov/gis/forecast/archive'
                                         ]
        self.gdf_names   =  gdf_names  = {
                                            file_names[0] : ['track_line_gdf', 'cone_gdf', 'points_gdf'],
                                            file_names[1] : ['init_radii_gdf', 'fcst_radii_gdf'],
                                            file_names[2] : ['best_track_points_gdf', 'best_track_line_gdf', 'best_track_radii_gdf', 'best_track_swath_gdf'],
                                            file_names[3] : ['gtwo_areas_gdf', 'gtwo_lines_gdf', 'gtwo_points_gdf'],
                                            file_names[4] : ['wsp_34_gdf_points', 'wsp_50_gdf_points', 'wsp_64_gdf_points'],
                                            file_names[5] : ['wsp_34_gdf_polygons', 'wsp_50_gdf_polygons', 'wsp_64_gdf_polygons']
                                          }

    def gtw_gdf(self): 
        #print(f'{urls[3]}/{file_names[3]}')
        #          url = f'https://www.nhc.noaa.gov/xgtwo/{file_name}'
        # download the file contents in binary format
        r = requests.get(f'{self.urls[3]}/{self.file_names[3]}')
        # open method to open a file on your system and write the contents
        with open(f'nhc_latest/{self.file_names[3]}', 'wb') as code:
            code.write(r.content)
        with ZipFile(f'nhc_latest/{self.file_names[3]}', 'r') as gtwo_zip:
            #print(gtwo_zip.namelist())
            gtwo_areas = gtwo_zip.namelist()[2]
            gtwo_lines = gtwo_zip.namelist()[7]
            gtwo_points = gtwo_zip.namelist()[12]
        gtwo_areas_gdf  = geopandas.read_file(f'zip://./nhc_latest/{self.file_names[3]}!{gtwo_areas}')
        gtwo_lines_gdf  = geopandas.read_file(f'zip://./nhc_latest/{self.file_names[3]}!{gtwo_lines}')
        gtwo_points_gdf = geopandas.read_file(f'zip://./nhc_latest/{self.file_names[3]}!{gtwo_points}')
        return gtwo_areas_gdf, gtwo_lines_gdf, gtwo_points_gdf
        
nch_dl       = NhcDownloaderBotNEW()
nch_dl.gtw_gdf()[0]



# a = nch_dl.file_names
# a
# gtwo_areas_gdf = nch_dl.gtw_gdf()[0]
# gtwo_areas_gdf
            


,BASIN,AREA,PROB2DAY,RISK2DAY,PROB5DAY,RISK5DAY,geometry
0,East Pacific,1,0%,Low,30%,Low,"POLYGON ((-96.10000 14.62000, -95.85843 14.389..."
